# Creación de paquetes ICMP

### Descripción: 
Con este código se va a intentar crear diferentes tipos de paquetes ICMP, mediante IA generativa. Se pretende pasar una descripción de estos paquetes al modelo y que este sea capaz de crear todos los campos necesarios para replicar un paquete con esas mismas especificaciones.

In [ ]:
import random
import re
import pickle
import os

import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = ''
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP, ICMP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de un paquete ICMP

<center><img src="https://upload.wikimedia.org/wikipedia/commons/7/70/ICMPv1.PNG">
    
<center> Esta imagen es genérica.


In [6]:
system_message = "In the next lines, I'll provide you with some packet descriptions for different types of ICMP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.\n\n"

# Echo ping request
system_message += "This is a ICMP echo ping (type 8) packet summary:\n"
system_message += "192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=18917, ttl=56\n"
system_message += "This is the python code to generate the packet:\n"
system_message += 'scapy.IP(src="192.168.1.194", dst="192.168.1.190", ttl=56)/scapy.ICMP(type=8, id=0x9015, seq=18917)\n'
system_message += "TTL definition ALWAYS goes on the TCP description.\n\n"

# Destination unreachable
system_message += "This is a ICMP destination unreachable (type 3) packet summary:\n"
system_message += "193.24.225.56 → 193.24.225.54 ICMP 74 Destination unreachable (Host unreachable)\n"
system_message += "This is the python code to generate the packet:\n"
system_message += 'scapy.IP(src="193.24.225.5", dst="193.24.225.54")/scapy.ICMP(type=3, code=1)\n'
system_message += "Take into account that the 'code' value depends on the reason of the destination unreachable, other values can be: 0=(Network unreachable), 1=(Host unreachable), 2=(Protocol unreachable), etc.\n\n"

# Time Exceeded
system_message += "This is a ICMP time exceeded (type 11) packet summary:\n"
system_message += "87.230.114.4 → 85.215.94.29 ICMP 72 Time-to-live exceeded (Time to live exceeded in transit)\n"
system_message += "This is the python code to generate the packet:\n"
system_message += 'scapy.IP(src="87.230.114.4", dst="85.215.94.29")/scapy.ICMP(type=11, code=0)\n'
system_message += "Take into account that the 'code' value depends on the reason of the Time-to-live exceeded, values can be: 0=(Time to live exceeded in transit), 1=(Fragment Reassembly Time Exceeded).\n\n"

system_message += "The number that appears right after to ICMP (100, 74 & 72 in the 3 summaries given as example) in every packet summary is the length of the packet, NOT the ICMP type. Using this as ICMP type is a flagrant error that must NOT happen\n"
system_message += "Take yout time (few seconds) to generate the commands, take especial care of the 'type' variable, as it must be coherent with the packet type (8 for Echo, 3 for Destination Unreachable, 0 for Echo Reply, etc\n\n"

In [8]:
print(system_message)

In the next lines, I'll provide you with some packet descriptions for different types of ICMP packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

This is a ICMP echo ping (type 8) packet summary:
192.168.1.194 → 192.168.1.190 ICMP 100 Echo (ping) request id=0x9015, seq=18917, ttl=56
This is the python code to generate the packet:
scapy.IP(src="192.168.1.194", dst="192.168.1.190", ttl=56)/scapy.ICMP(type=8, id=0x9015, seq=18917)
TTL definition ALWAYS goes on the TCP description.

This is a ICMP destination unreachable (type 3) packet summary:
193.24.225.56 → 193.24.225.54 ICMP 74 Destination unreachable (Host unreachable)
This is the python code to generate the packet:
scapy.IP(src="193.24.225.5", dst="193.24.225.54")/scapy.ICMP(type=3, 

## Carga de posibles descripciones

In [11]:
with open("../../data/text/ICMP_summaries_CORRECTED.txt","r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

1271

### Carga de posibles conversaciones anteriores para concatenarlas

In [14]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [16]:
try:
    responses = pickle.load( open("../../data/Packets/ICMP/pickle/ICMP_packet_pairs_Aday.pkl", "rb" ))
except:
    responses = []

## Generación de comandos de paquetes conversaciones ICMP

In [19]:
summaries_per_request = 10

In [24]:
for i in range(11):
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(i+1) +")")
    
    summaries = random.choices(packets_summary,k=summaries_per_request)
    text_sum = ""
    
    query_content = "\nBelow, I'll give you the descriptions of the packets you have to generate, the previous where just examples\n\n"
    
    for summary in summaries:
        query_content += "This is a packet summary:\n"
        query_content += summary + "\n"
        text_sum += summary + "\n"
        
    query_content += "\nGenerate python code for creating JUST the packets of the summaries given with scapy framework and put them all in a list named pkt_list. DO NOT create commands for other types of ICMP rather than the ones belonging to the packets wanted to generate.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.\n"
    
    print(query_content)
    
    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": query_content}]
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)

    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    codigo = responses[-1].completion.choices[0].message.content
    
    if codigo.startswith("```python"):
        codigo = codigo[len("```python"):].lstrip()  # Elimina ```python al inicio
    if codigo.endswith("```"):
        codigo = codigo[:-len("```")].rstrip()  # Elimina ``` al final
    
    exec(codigo)
    
    with open("../../data/Packets/ICMP/pcap/ICMPgenerated_gpt4omini.pcap", "ba+") as f:
        wrpcap(f, pkt_list, append=True)
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Packets/ICMP/pickle/ICMP_packet_pairs_Aday.pkl", "wb"))
    
    
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)

Below, I'll give you the descriptions of the packets you have to generate, the previous where just examples

This is a packet summary:
5.35.226.136          192.168.42.12         ICMP     110    Echo (ping) reply    id=0x0456, seq=246/62976, ttl=54
This is a packet summary:
5.35.226.136          192.168.42.12         ICMP     110    Echo (ping) reply    id=0x0456, seq=163/41728, ttl=54
This is a packet summary:
192.168.42.12         5.35.226.136          ICMP     110    Echo (ping) request  id=0x0456, seq=42/10752, ttl=64
This is a packet summary:
5.35.226.136          192.168.42.11         ICMP     110    Echo (ping) reply    id=0x0455, seq=298/10753, ttl=54
This is a packet summary:
5.35.226.136          192.168.42.12         ICMP     110    Echo (ping) reply    id=0x0456, seq=264/2049, ttl=54
This is a packet summary:
5.35.226.136          192.168.42.12         ICMP     110    

In [162]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

87.230.114.222        85.215.94.29          ICMP     72     Time-to-live exceeded (Time to live exceeded in transit)
192.168.121.2         192.168.120.1         ICMP     64     Timestamp request    id=0x0002, seq=1/256, ttl=255


pkt_list = []

# ICMP time exceeded packet
pkt_list.append(scapy.IP(src="87.230.114.222", dst="85.215.94.29")/scapy.ICMP(type=11, code=0))

# ICMP timestamp request packet
pkt_list.append(scapy.IP(src="192.168.121.2", dst="192.168.120.1", ttl=255)/scapy.ICMP(type=13, id=0x0002, seq=1)/scapy.Raw(load="\x01\x00\x00\x00\x00\x00\x00\x00"))

# ICMP echo reply packet
pkt_list.append(scapy.IP(src="192.168.1.190", dst="192.168.1.194")/scapy.ICMP(type=0, id=0x9015, seq=18917)/scapy.Raw(load="Hello, world!"))

# ICMP destination unreachable packet
pkt_list.append(scapy.IP(src="193.24.225.56", dst="193.24.225.54")/scapy.ICMP(type=3, code=1)/scapy.Raw(load="Host unreachable"))

# ICMP source quench packet
pkt_list.append(scapy.IP(src="192.168.1.1", dst="192.168.1.2")/scap

### Generación de paquetes

In [33]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<IP  frag=0 ttl=64 proto=icmp src=192.168.42.12 dst=5.35.226.136 |<ICMP  type=echo-request id=0x456 seq=0x123 |<Raw  load='\x00\x00\x00\x00\x00\x00\x00\x00' |>>>, <IP  frag=0 ttl=64 proto=icmp src=192.168.42.12 dst=5.35.226.136 |<ICMP  type=echo-request id=0x456 seq=0x9c |<Raw  load='\x00\x00\x00\x00\x00\x00\x00\x00' |>>>, <IP  frag=0 ttl=64 proto=icmp src=192.168.42.12 dst=5.35.226.136 |<ICMP  type=echo-request id=0x456 seq=0x115 |<Raw  load='\x00\x00\x00\x00\x00\x00\x00\x00' |>>>, <IP  frag=0 ttl=255 proto=icmp src=192.168.121.2 dst=192.168.120.1 |<ICMP  type=timestamp-request id=0x5 seq=0x4 |<Raw  load='\x00\x00\x00\x00\x00\x00\x00\x00' |>>>, <IP  frag=0 ttl=54 proto=icmp src=5.35.226.136 dst=192.168.42.12 |<ICMP  type=echo-reply id=0x456 seq=0xea |<Raw  load='\x00\x00\x00\x00\x00\x00\x00\x00' |>>>, <IP  frag=0 ttl=64 proto=icmp src=192.168.3.1 dst=192.168.3.85 |<ICMP  type=time-exceeded code=ttl-zero-during-transit |<Raw  load='\x00\x00\x00\x00\x00\x00\x00\x00' |>>>, <IP  frag=0 t

In [ ]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)

### Introducción en un pcap

In [175]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Packets/ICMP/pcap/ICMPgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)